In [47]:
import numpy as np
import pandas as pd
import datetime
import easygui
import time

In [48]:
# fname = easygui.filechoice(msg = '请点选作业完成时间统计表.xlsx')
fname = r'D:\a00nutstore\堂堂博瑞\七年级八班学生作业完成时间统计表20220922.xlsx'


In [53]:
def judgePeriod(x,timeStr):
    today = x[:10]
    weekday = datetime.date.weekday(datetime.datetime.strptime(today,'%Y-%m-%d'))
    if 'nan' in x :
        
        if weekday not in [4,5,6]  :
            return '待提交' 
        else :
            if weekday == 7 :
                return '待提交' 
            else :
                return "weekend"  
    elif '待提交'  in x :
             return  '待提交' 
    
        
    else :
        if today == timeStr:
            if f'{today} 00:00' <= x < f'{today} 22:00':
                return '22:00以前'
            elif f'{today} 22:00' <= x < f'{today} 23:00':
                return '22:00-23:00'
            elif f'{today} 23:00' <= x < f'{today} 24:00':
                return '23:00-24:00'
        else :
            return '24:00以后'
    return x


In [54]:
def getEndtime(timeStr):
    today = datetime.datetime.strptime(timeStr,'%Y-%m-%d')
    tomorrow = today + datetime.timedelta(days = 1)
    tomorrow_str = datetime.datetime.strftime(tomorrow,'%Y-%m-%d')
    print(tomorrow_str)
    return tomorrow_str

def getEachEndtime(df,columns):
    start = columns.index('学生姓名') 
    nums = [i for i in columns[start+1:]]
    news = []
    for num in nums:
        # 格式化为时间元组
        timeArray = time.localtime((num-25569) * 86400.0)
        # 将时间元组转换为时间字符串
        timeStr = time.strftime('%Y-%m-%d',timeArray)
        news.append(timeStr)
    columns = ['学生姓名'] + news
    df.columns = columns
    for timeStr in news:
        tomorrow_str = getEndtime(timeStr)
        df[f'{timeStr}'] = df[f'{timeStr}'].astype('str').str[:5]
        df[f'{timeStr}'] = np.where(df[f'{timeStr}'].str[:2].isin(['00','01','02','03','04','05','06']),tomorrow_str + ' '+ df[f'{timeStr}']  ,timeStr + ' '+ df[f'{timeStr}'])
    return news,df



In [55]:
dfs = pd.read_excel(fname,sheet_name=None)
dfs_period = []
dfs_names = []
for sheet_name,df in dfs.items():
    if sheet_name == '汇总' :
        continue
    else :
        columns = df.columns.to_list()
        for j in columns:
            if j in ['学号','备注']:
                columns.remove(j)
        df = df[columns]
        news,df = getEachEndtime(df,columns)
        print('df',df)
        dfs_names.append(df)
        for timeStr in news:
            df2 = df.assign(period = df[f'{timeStr}'].map(lambda x:judgePeriod(x,timeStr)))
            gp = df2.groupby('period',sort = False).count()
            gp = pd.DataFrame(gp.loc[:,f'{timeStr}'])
            gp = gp.loc[gp.index != 'weekend']
            gp.at['合计',f'{timeStr}'] = gp[f'{timeStr}'].sum()
            
            dfs_period.append(gp)


    

  
     
    



2022-09-14
df    学生姓名        2022-09-13
0   范绫君  2022-09-13 20:30
1   张博瑞  2022-09-13 22:07
2   石小凡  2022-09-13 22:21
3    涂佳  2022-09-13 22:21
4   刘翼虎  2022-09-13 22:25
5   吉聆萱  2022-09-13 22:27
6   李沛玲  2022-09-13 22:28
7   蔡辰阳  2022-09-13 22:30
8   贺文歆  2022-09-13 22:30
9   夏爱凡  2022-09-13 22:30
10  李堪越  2022-09-13 22:33
11  孙鸣谦  2022-09-13 22:35
12  魏志呈  2022-09-13 22:40
13  孙羽恬  2022-09-13 22:41
14  李弋戬  2022-09-13 22:45
15  谢雨涵  2022-09-13 22:49
16  张轩诚  2022-09-13 22:55
17  李默涵  2022-09-13 23:00
18  张心澄  2022-09-13 23:00
19  牟柏闻  2022-09-13 23:01
20  徐天怡  2022-09-13 23:01
21  孟子涵  2022-09-13 23:10
22  刘亦宸  2022-09-13 23:13
23  王子熠  2022-09-13 23:14
24  龚铂雅  2022-09-13 23:15
25  张杨洪  2022-09-13 23:16
26  周子循  2022-09-13 23:18
27  陈博文  2022-09-13 23:20
28  王梓骁  2022-09-13 23:20
29  吴瑞丰  2022-09-13 23:20
30  杨志博  2022-09-13 23:20
31  吴嘉鸿  2022-09-13 23:22
32  张芮程  2022-09-13 23:23
33  王欣颜  2022-09-13 23:25
34  朱彦熹  2022-09-13 23:25
35  熊峰义  2022-09-13 23:30
36  韩璐羽  2022-09-13 23:3

C:\temp\ipykernel_14916\2525036445.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{timeStr}'] = df[f'{timeStr}'].astype('str').str[:5]
C:\temp\ipykernel_14916\2525036445.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{timeStr}'] = np.where(df[f'{timeStr}'].str[:2].isin(['00','01','02','03','04','05','06']),tomorrow_str + ' '+ df[f'{timeStr}']  ,timeStr + ' '+ df[f'{timeStr}'])
C:\temp\ipykernel_14916\2525036445.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [57]:
for i in dfs_period:
    print(i)

             2022-09-13
period                 
22:00以前             1.0
22:00-23:00        16.0
23:00-24:00        30.0
24:00以后             5.0
合计                 52.0
             2022-09-14
period                 
22:00以前             4.0
22:00-23:00        18.0
23:00-24:00        25.0
24:00以后             5.0
合计                 52.0
             2022-09-15
period                 
22:00以前             1.0
22:00-23:00        17.0
23:00-24:00        26.0
24:00以后             8.0
合计                 52.0
         2022-09-16
period             
22:00以前         1.0
合计              1.0
             2022-09-17
period                 
22:00以前             2.0
22:00-23:00         1.0
23:00-24:00         1.0
合计                  4.0
             2022-09-18
period                 
22:00以前            35.0
22:00-23:00         8.0
23:00-24:00         3.0
24:00以后             1.0
合计                 47.0
             2022-09-19
period                 
22:00以前             3.0
22:00-23:00        18.0
23:00-24

In [58]:
df_period_total = pd.concat(dfs_period,axis = 1)
df_period_total 

,2022-09-13,2022-09-14,2022-09-15,2022-09-16,2022-09-17,2022-09-18,2022-09-19,2022-09-20,2022-09-21
period,,,,,,,,,
22:00以前,1.0,4.0,1.0,1.0,2.0,35.0,3.0,1.0,1.0
22:00-23:00,16.0,18.0,17.0,NaN,1.0,8.0,18.0,11.0,5.0
23:00-24:00,30.0,25.0,26.0,NaN,1.0,3.0,26.0,32.0,28.0
24:00以后,5.0,5.0,8.0,NaN,NaN,1.0,5.0,7.0,18.0
合计,52.0,52.0,52.0,1.0,4.0,47.0,52.0,52.0,52.0
待提交,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [56]:
df_period_total = pd.concat(dfs_period,axis = 1)
df_period_total = df_period_total.reindex(['22:00以前','22:00-23:00','23:00-24:00','24:00以后','待提交','合计'])
df_period_total


,2022-09-13,2022-09-14,2022-09-15,2022-09-16,2022-09-17,2022-09-18,2022-09-19,2022-09-20,2022-09-21
period,,,,,,,,,
22:00以前,1.0,4.0,1.0,1.0,2.0,35.0,3.0,1.0,1.0
22:00-23:00,16.0,18.0,17.0,NaN,1.0,8.0,18.0,11.0,5.0
23:00-24:00,30.0,25.0,26.0,NaN,1.0,3.0,26.0,32.0,28.0
24点以后,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
待提交,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
合计,52.0,52.0,52.0,1.0,4.0,47.0,52.0,52.0,52.0


In [13]:
for index,df0 in enumerate(dfs_names):
    if index == 0:
        df = df0
    else :
        df = pd.merge(df,df0,left_on = '学生姓名',right_on = '学生姓名')

df

,学生姓名,2022-09-13,2022-09-14,2022-09-15,2022-09-16,2022-09-17,2022-09-18,2022-09-19,2022-09-20,2022-09-21
0,范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-16 nan,2022-09-17 nan,2022-09-18 20:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
1,张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-16 nan,2022-09-17 nan,2022-09-18 08:19,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
2,石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-16 20:30,2022-09-17 nan,2022-09-18 nan,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
3,涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-16 nan,2022-09-17 nan,2022-09-18 11:04,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
4,刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-16 nan,2022-09-17 nan,2022-09-18 21:30,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
5,吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-16 nan,2022-09-17 nan,2022-09-18 15:22,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
6,李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-16 nan,2022-09-17 nan,2022-09-18 09:18,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
7,蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-16 nan,2022-09-17 23:30,2022-09-18 nan,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
8,贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-16 nan,2022-09-17 nan,2022-09-18 18:05,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53
9,夏爱凡,2022-09-13 22:30,2022-09-14 23:00,2022-09-15 23:30,2022-09-16 nan,2022-09-17 nan,2022-09-18 23:16,2022-09-19 23:15,2022-09-20 23:15,2022-09-21 23:45


In [42]:
df2 = df.copy()
df2 = df2.set_index('学生姓名')
df2

,2022-09-13,2022-09-14,2022-09-15,2022-09-16,2022-09-17,2022-09-18,2022-09-19,2022-09-20,2022-09-21
学生姓名,,,,,,,,,
范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-16 nan,2022-09-17 nan,2022-09-18 20:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-16 nan,2022-09-17 nan,2022-09-18 08:19,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-16 20:30,2022-09-17 nan,2022-09-18 nan,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-16 nan,2022-09-17 nan,2022-09-18 11:04,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-16 nan,2022-09-17 nan,2022-09-18 21:30,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-16 nan,2022-09-17 nan,2022-09-18 15:22,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-16 nan,2022-09-17 nan,2022-09-18 09:18,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-16 nan,2022-09-17 23:30,2022-09-18 nan,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-16 nan,2022-09-17 nan,2022-09-18 18:05,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53


In [15]:
dels = []
columns = df2.columns.to_list()
for i in columns:
    today = datetime.datetime.strptime(i,'%Y-%m-%d')
    weekday = datetime.date.weekday(today)
    if weekday in [4,5,6]:
        dels.append(i)
    else:
        continue
for i in dels:
    columns.remove(i)
columns
    

['2022-09-13',
 '2022-09-14',
 '2022-09-15',
 '2022-09-19',
 '2022-09-20',
 '2022-09-21']

In [17]:
sorted(columns)

['2022-09-13',
 '2022-09-14',
 '2022-09-15',
 '2022-09-19',
 '2022-09-20',
 '2022-09-21']

In [19]:
columns = ['2022-09-13',
           
 '2022-09-21',
 '2022-09-14',
 '2022-09-15',
 '2022-09-19',
 '2022-09-20']
columns

['2022-09-13',
 '2022-09-21',
 '2022-09-14',
 '2022-09-15',
 '2022-09-19',
 '2022-09-20']

In [22]:
columns = sorted(columns)
columns

['2022-09-13',
 '2022-09-14',
 '2022-09-15',
 '2022-09-19',
 '2022-09-20',
 '2022-09-21']

In [21]:
columns

['2022-09-13',
 '2022-09-21',
 '2022-09-14',
 '2022-09-15',
 '2022-09-19',
 '2022-09-20']

In [44]:
df2 = df2.applymap(lambda x:x[11:])
def func(x):
    if 'nan' in x:
        return pd.NaT
    else :
        return x
df2 = df2.applymap(func)
df2


,2022-09-13,2022-09-14,2022-09-15,2022-09-16,2022-09-17,2022-09-18,2022-09-19,2022-09-20,2022-09-21
学生姓名,,,,,,,,,
范绫君,20:30,21:00,23:30,NaT,NaT,20:30,23:30,23:07,23:28
张博瑞,22:07,22:24,22:27,NaT,NaT,08:19,22:47,22:48,23:10
石小凡,22:21,22:00,22:55,20:30,NaT,NaT,22:00,22:01,00:00
涂佳,22:21,22:42,22:18,NaT,NaT,11:04,23:00,23:19,23:21
刘翼虎,22:25,22:30,23:05,NaT,NaT,21:30,22:38,22:39,23:38
吉聆萱,22:27,22:10,22:29,NaT,NaT,15:22,22:40,22:28,23:08
李沛玲,22:28,22:22,23:10,NaT,NaT,09:18,22:18,23:30,23:45
蔡辰阳,22:30,23:10,23:25,NaT,23:30,NaT,22:59,23:30,00:34
贺文歆,22:30,23:16,22:50,NaT,NaT,18:05,22:16,23:50,22:53


In [ ]:
df_time = df_time.applymap(func)

In [23]:
df_time = df2[columns]
df_time

,2022-09-13,2022-09-14,2022-09-15,2022-09-19,2022-09-20,2022-09-21
学生姓名,,,,,,
范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53


In [24]:
def func(x):
    if 'nan' in x:
        return pd.NaT
    else :
        return x
df_time = df_time.applymap(func)
df_time


,2022-09-13,2022-09-14,2022-09-15,2022-09-19,2022-09-20,2022-09-21
学生姓名,,,,,,
范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53


In [25]:
df_time.fillna(df_time.max(0),inplace = True)
df_time

C:\temp\ipykernel_14916\2827856809.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_time.fillna(df_time.max(0),inplace = True)


,2022-09-13,2022-09-14,2022-09-15,2022-09-19,2022-09-20,2022-09-21
学生姓名,,,,,,
范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53


In [ ]:
for i in df_time.columns.to_list():
    pd.to_datetime(df_time[f'{i}'] )

In [ ]:
df_time

In [27]:
df1 = df_time
df1

,2022-09-13,2022-09-14,2022-09-15,2022-09-19,2022-09-20,2022-09-21
学生姓名,,,,,,
范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53


In [28]:
df1 = df1.dropna()
df1

,2022-09-13,2022-09-14,2022-09-15,2022-09-19,2022-09-20,2022-09-21
学生姓名,,,,,,
范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53


In [29]:
a = df1['2022-09-20'].max()

In [30]:
df_time.fillna(a,inplace = True)
df_time

,2022-09-13,2022-09-14,2022-09-15,2022-09-19,2022-09-20,2022-09-21
学生姓名,,,,,,
范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53


In [31]:
def func(x):
    x = x[-5:]
    hour,minute = x.split(':')
    if hour in ['00','01','02','03','04','05','06','07']:
        hour = int(hour) + 24
    return hour,int(minute)


In [32]:
df_hour = df_time.applymap(lambda  x :func(x)[0])
df_hour['mean'] = df_hour.astype(int).mean(1)
df_hour['hour0'] = df_hour['mean'].apply(lambda x:int(x))
df_hour['minute'] =60*(df_hour['mean']-df_hour['hour0'])
df_hour = df_hour[['hour0','minute']]
df_hour


,hour0,minute
学生姓名,,
范绫君,22,10.0
张博瑞,22,10.0
石小凡,22,20.0
涂佳,22,30.0
刘翼虎,22,20.0
吉聆萱,22,10.0
李沛玲,22,30.0
蔡辰阳,22,50.0
贺文歆,22,20.0


In [33]:
df_minute = df_time.applymap(lambda  x :func(x)[1])
df_minute['minute'] = df_minute.astype(int).mean(1)
df_minute = df_minute[['minute']]
df_minute

,minute
学生姓名,
范绫君,20.833333
张博瑞,27.166667
石小凡,12.833333
涂佳,20.166667
刘翼虎,29.166667
吉聆萱,23.666667
李沛玲,25.500000
蔡辰阳,31.333333
贺文歆,35.833333


In [34]:
df_hf = pd.merge(df_hour,df_minute,how = 'inner',left_on = '学生姓名',right_on = '学生姓名')
df_hf['minute_xy'] = round(df_hf['minute_x'] + df_hf['minute_y'],0)
df_hf['hour1'] = np.where(df_hf.minute_xy >= 60,1,0)
df_hf['hour'] = df_hf['hour0'] + df_hf['hour1']
df_hf['minute'] = np.where(df_hf.minute_xy < 60,df_hf.minute_xy,df_hf.minute_xy -60)
# df_hour = df_hour.astype({'hour':int,'minute':int})
df_hf = df_hf[['hour','minute']]
df_hf

,hour,minute
学生姓名,,
范绫君,22,31.0
张博瑞,22,37.0
石小凡,22,33.0
涂佳,22,50.0
刘翼虎,22,49.0
吉聆萱,22,34.0
李沛玲,22,56.0
蔡辰阳,23,21.0
贺文歆,22,56.0


In [35]:
def strfill(x):
    x = str(int(x)).zfill(2)
    return x
    
a = 1.0
b = strfill(a)
b

'01'

In [36]:
df_hf = df_hf.applymap(strfill)
df_hf 

,hour,minute
学生姓名,,
范绫君,22,31
张博瑞,22,37
石小凡,22,33
涂佳,22,50
刘翼虎,22,49
吉聆萱,22,34
李沛玲,22,56
蔡辰阳,23,21
贺文歆,22,56


In [37]:
df_hf['end_time'] = df_hf['hour'] + ':' + df_hf['minute']


In [38]:
df_hf = df_hf[['end_time']]
df_hf = df_hf.sort_values('end_time')
df_hf = df_hf.reset_index()
df_hf

,学生姓名,end_time
0,陈声扬,22:14
1,谢雨涵,22:15
2,吴嘉鸿,22:24
3,范绫君,22:31
4,石小凡,22:33
5,吉聆萱,22:34
6,张博瑞,22:37
7,孙鸣谦,22:38
8,赵怡楠,22:42
9,刘翼虎,22:49


In [46]:
with pd.ExcelWriter('1111.xlsx', engine='openpyxl')  as writer:
    df_period_total.to_excel(writer,sheet_name = '时间段')
    df2.to_excel(writer,sheet_name = '汇总')
    df_hf.to_excel(writer,sheet_name = '排名')
    

In [ ]:
df_hf.to_excel('七8班作业速度排名20220925.xlsx')

In [ ]:
lst = range(1,len(df_hf)+1)
lst

In [ ]:
df_hf = pd.read_clipboard()
df_hf.index = range(1,len(df_hf)+1)
df_hf.index.name = '排名'
df_hf

In [ ]:
df_hf.to_excel('七8班作业速度排名20220925.xlsx')

In [ ]:
df_hf = df_hf.reindex(range(1,len(df_hf)+1))
df_hf
              

In [61]:
import numpy as np
import pandas as pd
import datetime
import time
import easygui
import os


def judgePeriod(x,timeStr):
    today = x[:10]
    weekday = datetime.date.weekday(datetime.datetime.strptime(today,'%Y-%m-%d'))
    if 'nan' in x :
        
        if weekday not in [4,5,6]  :
            return '待提交'
        else :
            if weekday == 7 :
                return '待提交'
            else :
                return "weekend"  
    elif '待提交'  in x :
             return  '待提交'
    else :
        if today == timeStr:
            if f'{today} 00:00' <= x < f'{today} 22:00':
                return '22:00以前'
            elif f'{today} 22:00' <= x < f'{today} 23:00':
                return '22:00-23:00'
            elif f'{today} 23:00' <= x < f'{today} 24:00':
                return '23:00-24:00'
        else :
            return '24:00以后'
    return x


def getEndtime(timeStr):
    today = datetime.datetime.strptime(timeStr,'%Y-%m-%d')
    tomorrow = today + datetime.timedelta(days = 1)
    tomorrow_str = datetime.datetime.strftime(tomorrow,'%Y-%m-%d')
    print(tomorrow_str)
    return tomorrow_str

def getEachEndtime(df,columns):
    start = columns.index('学生姓名')
    nums = [i for i in columns[start+1:]]
    news = []
    for num in nums:
        # 格式化为时间元组
        timeArray = time.localtime((num-25569) * 86400.0)
        # 将时间元组转换为时间字符串
        timeStr = time.strftime('%Y-%m-%d',timeArray)
        news.append(timeStr)
    columns = ['学生姓名'] + news
    df.columns = columns
    for timeStr in news:
        tomorrow_str = getEndtime(timeStr)
        df[f'{timeStr}'] = df[f'{timeStr}'].astype('str').str[:5]
        df[f'{timeStr}'] = np.where(df[f'{timeStr}'].str[:2].isin(['00','01','02','03','04','05','06']),tomorrow_str + ' '+ df[f'{timeStr}']  ,timeStr + ' '+ df[f'{timeStr}'])
    return news,df

def getNat(x):
    if 'nan' in x:
        return pd.NaT
    else:
        return x

def hourMinute(x):
    x = x[-5:]
    hour, minute = x.split(':')
    if hour in ['00', '01', '02', '03', '04', '05', '06', '07']:
        hour = int(hour) + 24
    return hour, int(minute)

def strfill(x):
    x = str(int(x)).zfill(2)
    return x


fname = easygui.fileopenbox(msg = '请点选作业完成时间统计表.xlsx')
fname1 = easygui.fileopenbox(msg = '请点选七8班名单.xlsx')
df_name = pd.read_excel(fname1)
dfs = pd.read_excel(fname,sheet_name=None)
dfs_period = []
dfs_names = []
for sheet_name,df in dfs.items():
    if sheet_name == '汇总' :
        continue
    else :
        columns = df.columns.to_list()
        for j in columns:
            if j in ['学号','备注']:
                columns.remove(j)
        df = df[columns]
        news,df = getEachEndtime(df,columns)
        print('df',df)
        dfs_names.append(df)
        for timeStr in news:
            df2 = df.assign(period = df[f'{timeStr}'].map(lambda x:judgePeriod(x,timeStr)))
            gp = df2.groupby('period',sort = False).count()
            gp = pd.DataFrame(gp.loc[:,f'{timeStr}'])
            gp = gp.loc[gp.index != 'weekend']
            gp.at['合计',f'{timeStr}'] = gp[f'{timeStr}'].sum()
            dfs_period.append(gp)

df_period_total = pd.concat(dfs_period,axis = 1)
df_period_total = df_period_total.reindex(['22:00以前','22:00-23:00','23:00-24:00','24:00以后','待提交','合计'])
df_period_total.index.name = '完成作业时间段'

for index,df0 in enumerate(dfs_names):
    if index == 0:
        df = df0
    else :
        df = pd.merge(df,df0,left_on = '学生姓名',right_on = '学生姓名')

df2 = df.copy()
df2


2022-09-14
df    学生姓名        2022-09-13
0   范绫君  2022-09-13 20:30
1   张博瑞  2022-09-13 22:07
2   石小凡  2022-09-13 22:21
3    涂佳  2022-09-13 22:21
4   刘翼虎  2022-09-13 22:25
5   吉聆萱  2022-09-13 22:27
6   李沛玲  2022-09-13 22:28
7   蔡辰阳  2022-09-13 22:30
8   贺文歆  2022-09-13 22:30
9   夏爱凡  2022-09-13 22:30
10  李堪越  2022-09-13 22:33
11  孙鸣谦  2022-09-13 22:35
12  魏志呈  2022-09-13 22:40
13  孙羽恬  2022-09-13 22:41
14  李弋戬  2022-09-13 22:45
15  谢雨涵  2022-09-13 22:49
16  张轩诚  2022-09-13 22:55
17  李默涵  2022-09-13 23:00
18  张心澄  2022-09-13 23:00
19  牟柏闻  2022-09-13 23:01
20  徐天怡  2022-09-13 23:01
21  孟子涵  2022-09-13 23:10
22  刘亦宸  2022-09-13 23:13
23  王子熠  2022-09-13 23:14
24  龚铂雅  2022-09-13 23:15
25  张杨洪  2022-09-13 23:16
26  周子循  2022-09-13 23:18
27  陈博文  2022-09-13 23:20
28  王梓骁  2022-09-13 23:20
29  吴瑞丰  2022-09-13 23:20
30  杨志博  2022-09-13 23:20
31  吴嘉鸿  2022-09-13 23:22
32  张芮程  2022-09-13 23:23
33  王欣颜  2022-09-13 23:25
34  朱彦熹  2022-09-13 23:25
35  熊峰义  2022-09-13 23:30
36  韩璐羽  2022-09-13 23:3

C:\temp\ipykernel_14916\2065988538.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{timeStr}'] = df[f'{timeStr}'].astype('str').str[:5]
C:\temp\ipykernel_14916\2065988538.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{timeStr}'] = np.where(df[f'{timeStr}'].str[:2].isin(['00','01','02','03','04','05','06']),tomorrow_str + ' '+ df[f'{timeStr}']  ,timeStr + ' '+ df[f'{timeStr}'])
C:\temp\ipykernel_14916\2065988538.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

2022-09-19
df    学生姓名        2022-09-16        2022-09-17        2022-09-18
0   石小凡  2022-09-16 20:30    2022-09-17 nan    2022-09-18 nan
1   韩璐羽    2022-09-16 nan  2022-09-17 19:58    2022-09-18 nan
2   张轩诚    2022-09-16 nan  2022-09-17 20:46    2022-09-18 nan
3   曹恩睿    2022-09-16 nan  2022-09-17 22:20    2022-09-18 nan
4   蔡辰阳    2022-09-16 nan  2022-09-17 23:30    2022-09-18 nan
5   赵怡楠    2022-09-16 nan    2022-09-17 nan  2022-09-18 07:43
6   张博瑞    2022-09-16 nan    2022-09-17 nan  2022-09-18 08:19
7   朱彦熹    2022-09-16 nan    2022-09-17 nan  2022-09-18 09:00
8   李沛玲    2022-09-16 nan    2022-09-17 nan  2022-09-18 09:18
9   刘亦宸    2022-09-16 nan    2022-09-17 nan  2022-09-18 09:55
10  吴嘉鸿    2022-09-16 nan    2022-09-17 nan  2022-09-18 10:04
11  谢雨涵    2022-09-16 nan    2022-09-17 nan  2022-09-18 10:05
12  程致行    2022-09-16 nan    2022-09-17 nan  2022-09-18 10:48
13   涂佳    2022-09-16 nan    2022-09-17 nan  2022-09-18 11:04
14  孟子涵    2022-09-16 nan    2022-09-17 nan  2022-09-18 

,学生姓名,2022-09-13,2022-09-14,2022-09-15,2022-09-16,2022-09-17,2022-09-18,2022-09-19,2022-09-20,2022-09-21
0,范绫君,2022-09-13 20:30,2022-09-14 21:00,2022-09-15 23:30,2022-09-16 nan,2022-09-17 nan,2022-09-18 20:30,2022-09-19 23:30,2022-09-20 23:07,2022-09-21 23:28
1,张博瑞,2022-09-13 22:07,2022-09-14 22:24,2022-09-15 22:27,2022-09-16 nan,2022-09-17 nan,2022-09-18 08:19,2022-09-19 22:47,2022-09-20 22:48,2022-09-21 23:10
2,石小凡,2022-09-13 22:21,2022-09-14 22:00,2022-09-15 22:55,2022-09-16 20:30,2022-09-17 nan,2022-09-18 nan,2022-09-19 22:00,2022-09-20 22:01,2022-09-22 00:00
3,涂佳,2022-09-13 22:21,2022-09-14 22:42,2022-09-15 22:18,2022-09-16 nan,2022-09-17 nan,2022-09-18 11:04,2022-09-19 23:00,2022-09-20 23:19,2022-09-21 23:21
4,刘翼虎,2022-09-13 22:25,2022-09-14 22:30,2022-09-15 23:05,2022-09-16 nan,2022-09-17 nan,2022-09-18 21:30,2022-09-19 22:38,2022-09-20 22:39,2022-09-21 23:38
5,吉聆萱,2022-09-13 22:27,2022-09-14 22:10,2022-09-15 22:29,2022-09-16 nan,2022-09-17 nan,2022-09-18 15:22,2022-09-19 22:40,2022-09-20 22:28,2022-09-21 23:08
6,李沛玲,2022-09-13 22:28,2022-09-14 22:22,2022-09-15 23:10,2022-09-16 nan,2022-09-17 nan,2022-09-18 09:18,2022-09-19 22:18,2022-09-20 23:30,2022-09-21 23:45
7,蔡辰阳,2022-09-13 22:30,2022-09-14 23:10,2022-09-15 23:25,2022-09-16 nan,2022-09-17 23:30,2022-09-18 nan,2022-09-19 22:59,2022-09-20 23:30,2022-09-22 00:34
8,贺文歆,2022-09-13 22:30,2022-09-14 23:16,2022-09-15 22:50,2022-09-16 nan,2022-09-17 nan,2022-09-18 18:05,2022-09-19 22:16,2022-09-20 23:50,2022-09-21 22:53
9,夏爱凡,2022-09-13 22:30,2022-09-14 23:00,2022-09-15 23:30,2022-09-16 nan,2022-09-17 nan,2022-09-18 23:16,2022-09-19 23:15,2022-09-20 23:15,2022-09-21 23:45


In [62]:
df3 = df2.applymap(lambda x:x[11:])
df3

,学生姓名,2022-09-13,2022-09-14,2022-09-15,2022-09-16,2022-09-17,2022-09-18,2022-09-19,2022-09-20,2022-09-21
0,,20:30,21:00,23:30,nan,nan,20:30,23:30,23:07,23:28
1,,22:07,22:24,22:27,nan,nan,08:19,22:47,22:48,23:10
2,,22:21,22:00,22:55,20:30,nan,nan,22:00,22:01,00:00
3,,22:21,22:42,22:18,nan,nan,11:04,23:00,23:19,23:21
4,,22:25,22:30,23:05,nan,nan,21:30,22:38,22:39,23:38
5,,22:27,22:10,22:29,nan,nan,15:22,22:40,22:28,23:08
6,,22:28,22:22,23:10,nan,nan,09:18,22:18,23:30,23:45
7,,22:30,23:10,23:25,nan,23:30,nan,22:59,23:30,00:34
8,,22:30,23:16,22:50,nan,nan,18:05,22:16,23:50,22:53
9,,22:30,23:00,23:30,nan,nan,23:16,23:15,23:15,23:45


In [60]:
df2 = df.applymap(lambda x:x[11:])
df2 = df2.applymap(getNat)
print('df2',df2)
print('df1',df1)
# df_nameLst = pd.merge(df1.iloc[:,1:],df2,how = 'inner',left_on = '学生姓名',right_on = '学生姓名')

# df = df.set_index('学生姓名')
# dels = []
# columns = df.columns.to_list()
# for i in columns:
#     today = datetime.datetime.strptime(i,'%Y-%m-%d')
#     weekday = datetime.date.weekday(today)
#     if weekday in [4,5,6]:
#         dels.append(i)
#     else:
#         continue
# for i in dels:
#     columns.remove(i)

# columns = sorted(columns)
# end_date = columns[-1]
# result_fname = f'七8班作业速度排名{end_date}.xlsx'
# df_time = df[columns]
# df_time = df_time.applymap(getNat)
# df1 = df_time.copy()
# df1 = df1.dropna()
# for i in columns:
#     a = df1[f'{i}'].max()
#     df_time[f'{i}'].fillna(a,inplace = True)
# df_hour = df_time.applymap(lambda  x :hourMinute(x)[0])
# df_hour['mean'] = df_hour.astype(int).mean(1)
# df_hour['hour0'] = df_hour['mean'].apply(lambda x:int(x))
# df_hour['minute'] =60*(df_hour['mean']-df_hour['hour0'])
# df_hour = df_hour[['hour0','minute']]
# df_minute = df_time.applymap(lambda  x :hourMinute(x)[1])
# df_minute['minute'] = df_minute.astype(int).mean(1)
# df_minute = df_minute[['minute']]
# df_hf = pd.merge(df_hour,df_minute,how = 'inner',left_on = '学生姓名',right_on = '学生姓名')
# df_hf['minute_xy'] = round(df_hf['minute_x'] + df_hf['minute_y'],0)
# df_hf['hour1'] = np.where(df_hf.minute_xy >= 60,1,0)
# df_hf['hour'] = df_hf['hour0'] + df_hf['hour1']
# df_hf['minute'] = np.where(df_hf.minute_xy < 60,df_hf.minute_xy,df_hf.minute_xy -60)
# df_hf = df_hf[['hour','minute']]
# df_hf = df_hf.applymap(strfill)
# df_hf['end_time'] = df_hf['hour'] + ':' + df_hf['minute']
# df_hf = df_hf[['end_time']]
# df_hf = df_hf.sort_values('end_time')
# df_hf = df_hf.reset_index()
# df_hf.index = range(1,len(df_hf)+1)
# df_hf.index.name = '排名'

# with pd.ExcelWriter(result_fname, engine='openpyxl')  as writer:
#     df_period_total.to_excel(writer,sheet_name = '时间段')
#     df_nameLst.to_excel(writer, sheet_name= '同学')
#     df_hf.to_excel(writer,sheet_name = '汇总')

# os.startfile(result_fname)










,七年级八班学生作业完成时间统计表,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,学号,学生姓名,44817,44818,44819,44820,44821,44822,44823,44824,...,完成作业时间段,44817.0,44818.0,44819.0,44820.0,44821.0,44822.0,44823.0,44824.0,44825.0
1,1,蔡辰阳,22:30:00,23:10,23:25,NaN,23:30,NaN,22:59,23:30,...,22：00以前,1.0,4.0,1.0,1.0,2.0,35.0,3.0,1.0,1.0
2,2,曹恩睿,23:46,23:12,23:25,NaN,22:20,NaN,23:30,00:11,...,22：00-23：00,16.0,18.0,17.0,NaN,1.0,8.0,18.0,11.0,5.0
3,3,陈博文,23:20,23:40,00:12,NaN,NaN,22:45,23:30,23:30,...,23：00-24：00,30.0,25.0,26.0,NaN,1.0,3.0,26.0,32.0,28.0
4,4,陈声扬,23:44,22:00,21:49,NaN,NaN,12:52,22:28,21:52,...,24：00以后,5.0,5.0,8.0,NaN,NaN,1.0,5.0,7.0,18.0
5,5,程致行,23:36,23:30,23:30,NaN,NaN,10:48,23:29,22:58,...,待提交,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,6,董予墨,23:50,22:40,23:10,NaN,NaN,17:15:00,23:10,23:15,...,合计,52.0,52.0,52.0,52.0,NaN,NaN,52.0,52.0,52.0
7,7,范绫君,20:30,21:00,22:30,NaN,NaN,20:30,23:30,23:07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,付乐莱,00:27,23:39,22:46,NaN,NaN,15:17,22:13,23:45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,高源翎,01:00,00:00,00:00,NaN,NaN,18:08:00,00:00,23:58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
